## ESA Project Results Repository (PRR) Data Access and Collections Preview

This notebook has been created to support the access to the users of EarthCODE and APEX, who would like to exploit available products and project results stored in the [ESA Project Results Repository (PRR)](https://eoresults.esa.int/). PRR provides access to data, workflows, experiments and documentation from ESA EOP-S Projects organised across Collections, accessible via [OGC Records](https://ogcapi.ogc.org/records) e S[TAC API](https://github.com/radiantearth/stac-api-spec).

Each collection contains [STAC Items](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md), with their related assets stored within the PRR storage.

Scientists/commercial companies can access the PRR via the [EarthCODE](https://earthcode.esa.int/) and [APEx](https://esa-apex.github.io/apex_documentation/) projects.

Use following notebook cells to preview the content of the ESA PRR and request the download of selected products. 

### Loading Libraries and set up logging level

In [1]:
import os
import logging
import pprint
import shutil
from urllib.parse import urljoin
from urllib.request import urlretrieve

#Make sure you have installed pystac_client before running this
from pystac_client import Client

# set pystac_client logger to DEBUG to see API calls
logging.basicConfig()
logger = logging.getLogger("pystac_client")
logger.setLevel(logging.DEBUG)


### Connect to ESA PRR Catalog and display the list of collections available

In [2]:
# URL of the STAC Catalog to query
catalog_url = "https://eoresults.esa.int/stac"

# custom headers
headers = []

cat = Client.open(catalog_url, headers=headers)
cat # display the basic informaiton about PRR Catalog in STAC Format

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


<Client id=stac-fastapi>

Use the cell below to access entire list of collections available in ESA PRR.

In [3]:
collection_search = cat.collection_search(limit=150)
print(f"Total number of collections found in ESA PRR is {collection_search.matched()}")

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/ Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections?limit=150 Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


Total number of collections found in ESA PRR is 30


In [4]:
# Display the name of the names of collection (collection-ids) to be used to filter the colleciton of interest
for collection in collection_search.collections_as_dicts():
    print(collection.get("id", "Unnamed Collection"))

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections?limit=150 Headers: {'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


APEX_TEST
EARTHCODE_TEST
ESA_LUISA_HANPP_HARV_CROPLAND
ESA_LUISA_HANPP_HARV_FOREST
ESA_LUISA_HANPP_HARV_RANGELAND
ESA_LUISA_HANPP_LUC
ESA_LUISA_NPP_ACT
ESA_PEOPLE_EA_COASTAL_CONDITIONS
ESA_PEOPLE_EA_EXTENT_MAPS
ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012
ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2021
ESA_PEOPLE_EA_WOOD_PROVISION
ESA_PEOPLE_ECOSCHEMES_CROP_DIVERSIFICATION
ESA_PEOPLE_ECOSCHEMES_CROP_EROSION
ESA_PEOPLE_ECOSCHEMES_CROP_ROTATION
ESA_PEOPLE_ECOSCHEMES_CROP_TYPE
ESAWAAI
ESA_WORLDCEREAL_ACTIVECROPLAND
ESA_WORLDCEREAL_IRRIGATION
ESA_WORLDCEREAL_MAIZE
ESA_WORLDCEREAL_SPRINGCEREALS
ESA_WORLDCEREAL_TEMPORARYCROPS
ESA_WORLDCEREAL_WINTERCEREALS
ESA_WORLDCOVER_10M_2020_V1
ESA_WORLDCOVER_10M_2021_V2
EXTRAIM_DAILY_PRECIPITATION
PRR_TEST
sentinel3-ampli-ice-sheet-elevation
worldsoils-soc


Alternatively, you can display the metadata of all STAC Collection available: 

In [5]:
# Or they can be displayed with their full metadata
collection_search = cat.collection_search(
    datetime='2023-04-02T00:00:00Z/2024-08-10T23:59:59Z',  #this is an additional filter to be added to filter the collections based on the date.
    limit=10
)
print(f"{collection_search.matched()} collections found")
print("PRR available Collections\n")

for results in collection_search.collections_as_dicts():  # maybe this part should not display entire dic
    pp = pprint.PrettyPrinter(depth=4)
    pp.pprint(results)

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections?limit=10&datetime=2023-04-02T00%3A00%3A00Z%2F2024-08-10T23%3A59%3A59Z Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections?limit=10&datetime=2023-04-02T00%3A00%3A00Z%2F2024-08-10T23%3A59%3A59Z Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}
DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections?limit=10&datetime=2023-04-02T00:00:00Z/2024-08-10T23:59:59Z&offset=10 Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


28 collections found
PRR available Collections

{'description': 'Just for testing, contains nothing relevant to the users',
 'extent': {'spatial': {'bbox': [[...]]}, 'temporal': {'interval': [[...]]}},
 'id': 'APEX_TEST',
 'license': 'Open Data',
 'links': [{'href': 'https://eoresults.esa.int/stac/collections/APEX_TEST/items',
            'rel': 'items',
            'type': 'application/geo+json'},
           {'href': 'https://eoresults.esa.int/stac/',
            'rel': 'parent',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac/',
            'rel': 'root',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac/collections/APEX_TEST',
            'rel': 'self',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac/collections/APEX_TEST/queryables',
            'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
            'title': 'Queryables',
            

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections?limit=10&datetime=2023-04-02T00:00:00Z/2024-08-10T23:59:59Z&offset=20 Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


{'description': 'Collection containing Habitat maps is input for Ecosystem '
                'Extent Accounts for a number of demonstrator countries. The '
                'Habitat Mapping within these datasets is based on the EUNIS '
                'habitat types hierarchical view of 2021 ( '
                'https://eunis.eea.europa.eu/habitats.jsp)',
 'extent': {'spatial': {'bbox': [[...]]}, 'temporal': {'interval': [[...]]}},
 'id': 'ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2021',
 'license': 'Open Data',
 'links': [{'href': 'https://eoresults.esa.int/stac/collections/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2021/items',
            'rel': 'items',
            'type': 'application/geo+json'},
           {'href': 'https://eoresults.esa.int/stac/',
            'rel': 'parent',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac/',
            'rel': 'root',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac/collectio

### Open Sentinel-3 AMPLI Ice Sheet Elevation collection

To access specific collection, we will use the *collection id* from the cell above. Type `sentinel3-ampli-ice-sheet-elevation` to connect to selected collection and display its metadata. 

In [6]:
collection = cat.get_collection("sentinel3-ampli-ice-sheet-elevation") # place here the id of the selected collection
#collection # or use simply json metadata to display the information 
print("PRR Sentinel-3 AMPLI Collection\n")
pp = pprint.PrettyPrinter(depth=4)
pp.pprint(collection.to_dict())

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections/sentinel3-ampli-ice-sheet-elevation Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


PRR Sentinel-3 AMPLI Collection

{'description': 'Ice sheet elevation estimated along the Sentinel-3 satellite '
                'track, as retrieved with the Altimeter data Modelling and '
                'Processing for Land Ice (AMPLI). The products cover '
                'Antarctica and Greenland.',
 'extent': {'spatial': {'bbox': [[...]]}, 'temporal': {'interval': [[...]]}},
 'id': 'sentinel3-ampli-ice-sheet-elevation',
 'license': 'CC-BY-4.0',
 'links': [{'href': 'https://eoresults.esa.int/stac/collections/sentinel3-ampli-ice-sheet-elevation/items',
            'rel': 'items',
            'type': 'application/geo+json'},
           {'href': 'https://eoresults.esa.int/stac/',
            'rel': 'parent',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac',
            'rel': 'root',
            'title': 'Project Results Repository',
            'type': 'application/json'},
           {'href': 'https://eoresults.esa.int/stac/collections/se

In [7]:
#Or display it in the STAC file format to better visualise the attributes and properties 
collection

<CollectionClient id=sentinel3-ampli-ice-sheet-elevation>

From the cell below, we will retrieve and explore **queryable fields** from a **STAC API**, which allows us to understand what parameters we can use for filtering our searches.

In [8]:
queryable = collection.get_queryables()

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(queryable)

DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections/sentinel3-ampli-ice-sheet-elevation/queryables Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


{'$id': 'https://eoresults.esa.int/collections/sentinel3-ampli-ice-sheet-elevation/queryables',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'additionalProperties': True,
 'properties': {'datetime': {'description': 'Datetime',
                             'format': 'date-time',
                             'pattern': '(\\+00:00|Z)$',
                             'title': 'Acquired',
                             'type': 'string'},
                'geometry': {'$ref': 'https://geojson.org/schema/Feature.json',
                             'description': 'Item Geometry',
                             'title': 'Item Geometry'},
                'id': {'$ref': 'https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json#/definitions/core/allOf/2/properties/id',
                       'description': 'Item identifier',
                       'title': 'Item ID'}},
 'title': 'STAC Queryables.',
 'type': 'object'}


### Display STAC Items from Sentinel-3 AMPLI Ice Sheet Elevation collection 

By executing the cell below you will get the ids of items that can be found in the specific collection (requested above) First five items from the list are printed out. 

In [9]:
items = collection.get_items()

# flush stdout so we can see the exact order that things happen
def get_five_items(items):
    for i, item in enumerate(items):
        print(f"{i}: {item}", flush=True)
        if i == 4:
            return
        
print("First page", flush=True)
get_five_items(items)

print("Second page", flush=True)
get_five_items(items)

First page


DEBUG:pystac_client.stac_api_io:GET https://eoresults.esa.int/stac/collections/sentinel3-ampli-ice-sheet-elevation/items?collections=sentinel3-ampli-ice-sheet-elevation Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}


0: <Item id=sentinel-3-ampli-user-handbook>
1: <Item id=sentinel-3b-antarctica-cycle092>
2: <Item id=sentinel-3b-greenland-cycle092>
3: <Item id=sentinel-3a-antarctica-cycle111>
4: <Item id=sentinel-3a-greenland-cycle111>
Second page
0: <Item id=sentinel-3b-antarctica-cycle091>
1: <Item id=sentinel-3b-greenland-cycle091>
2: <Item id=sentinel-3a-antarctica-cycle110>
3: <Item id=sentinel-3a-greenland-cycle110>
4: <Item id=sentinel-3b-antarctica-cycle090>


Now execute a **search with a set of parameters**. In this case it returns just one item because **we filter on one queryable parameter** `(id)`

In [10]:
#Search for items based on spatio-temporal properties

# AOI entire world
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-180, -90],
            [-180, 90],
            [180 , 90],
            [180, -90],
            [-180, -90],
        ]
    ],
}

# limit sets the # of items per page so we can see multiple pages getting fetched
#In this search we apply also filtering on ID that is one of the searchable parameters for the colletion
search = cat.search(
    max_items=7,
    limit=5,
    collections="sentinel3-ampli-ice-sheet-elevation",        # specify collection id
    intersects=geom,
    query={"id": {"eq": "sentinel-3a-antarctica-cycle107"}},  # search for the specific Item in the collection 
    datetime="2023-04-02T00:00:00Z/2024-08-10T23:59:59Z",     # specify thestart and end date of the time frame to perform the search 
)

items = list(search.items())

print(len(items))

pp = pprint.PrettyPrinter(depth=4)
pp.pprint([i.to_dict() for i in items])

DEBUG:pystac_client.stac_api_io:POST https://eoresults.esa.int/stac/search Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '300', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2023-04-02T00:00:00Z/2024-08-10T23:59:59Z", "collections": ["sentinel3-ampli-ice-sheet-elevation"], "intersects": {"type": "Polygon", "coordinates": [[[-180, -90], [-180, 90], [180, 90], [180, -90], [-180, -90]]]}, "query": {"id": {"eq": "sentinel-3a-antarctica-cycle107"}}}


1
[{'assets': {'S3A_SR_2_TDP_LI_20231216T230713_20231216T231833_20250416T220127_0680_107_001______CNE_ANT_V001': {'cycle_number': '107',
                                                                                                                'href': '/d/sentinel3-ampli-ice-sheet-elevation/2023/12/16/sentinel-3a-antarctica-cycle107/S3A_SR_2_TDP_LI_20231216T230713_20231216T231833_20250416T220127_0680_107_001______CNE_ANT_V001.nc',
                                                                                                                'orbit_direction': 'descending',
                                                                                                                'orbit_number': '40784',
                                                                                                                'relative_orbit_number': '001',
                                                                                                                'roles': ['data'],
   

If you do not know the item id, search trhough available mission platform, region, number of the cycle and the datetime range of the products of interest. <br>**You can specify them by filtering based on following possible values:**<br>
* missions: `3a` or `3b`
* regions: `anarctica` or `greenland`
* cycle range: for sentinel-3a possible cycle range is from `005 to 112`; while sentinel-3b has range from `011-093`
* datetime: specify the time frame of the products from the range between: `2016-06-01 00:00:00 UTC – 2024-05-09 00:00:00 UTC`

In [22]:
#Search for items from specific mission and type of the instrument (based on the id) and the region as well as cycle number 
# Define your cycle range and mission types
cycle_range = [f"{i:03d}" for i in range(90, 111)] #005 to 111   # for sentinel-3a possible cycle range is from 005 to 111; while s3b has range from 011-092
missions = ["3b"]          # select the mission and sensor type from:"sentinel-3a" or "sentinel-3b"]  
regions = ["antarctica"]              # specify the region from: "antarctica" or "greenland"

# AOI entire world
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-180, -90],
            [-180, 90],
            [180 , 90],
            [180, -90],
            [-180, -90],
        ]
    ],
}

# limit sets the # of items per page so we can see multiple pages getting fetched
#In this search we apply also filtering on ID that is one of the searchable parameters for the colletion
search = cat.search(
    max_items=7,
    limit=5,
    collections="sentinel3-ampli-ice-sheet-elevation",
    intersects=geom,  # search for the specific Item in the collection 
    datetime="2021-04-02T00:00:00Z/2024-08-10T23:59:59Z",     # specify the start and end date of the time frame to perform the search which are: 2016-06-01 00:00:00 UTC – 2024-05-09 00:00:00 UTC
)
items = list(search.items())
print(f"Number of items found: {len(items)}")
print(items)

pp = pprint.PrettyPrinter(depth=4)

filtered = [
    item for item in items
    if any(m in item.id.lower()  for m in missions)
    and any(r in item.id.lower()  for r in regions)
    and any(f"cycle{c}" in item.id.lower() for c in cycle_range)
]


#for i, item in enumerate(filtered, 2):
   # print(f"{i}. {item.id} @ {item.datetime}")

## Print number of filtered items
print(f"Number of filtered items: {len(filtered)}")
for i, item in enumerate(filtered, 2):
    print(f"{i}. {item.id} @ {item.datetime}")

DEBUG:pystac_client.stac_api_io:POST https://eoresults.esa.int/stac/search Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '240', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2021-04-02T00:00:00Z/2024-08-10T23:59:59Z", "collections": ["sentinel3-ampli-ice-sheet-elevation"], "intersects": {"type": "Polygon", "coordinates": [[[-180, -90], [-180, 90], [180, 90], [180, -90], [-180, -90]]]}}
DEBUG:pystac_client.stac_api_io:POST https://eoresults.esa.int/stac/search Headers: {'User-Agent': 'python-requests/2.32.4', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '325', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2021-04-02T00:00:00Z/2024-08-10T23:59:59Z", "collections": ["sentinel3-ampli-ice-sheet-elevation"], "intersects": {"type": "Polygon", "coordinates": [[[-180, -90

Number of items found: 7
[<Item id=sentinel-3b-antarctica-cycle092>, <Item id=sentinel-3b-greenland-cycle092>, <Item id=sentinel-3a-antarctica-cycle111>, <Item id=sentinel-3a-greenland-cycle111>, <Item id=sentinel-3b-antarctica-cycle091>, <Item id=sentinel-3b-greenland-cycle091>, <Item id=sentinel-3a-antarctica-cycle110>]
Number of filtered items: 2
2. sentinel-3b-antarctica-cycle092 @ 2024-04-12 23:09:27+00:00
3. sentinel-3b-antarctica-cycle091 @ 2024-03-16 23:09:24+00:00


## Download all assets from the selected item <br>
Based on the selection done in the previous cell, download the products to the `downloads` folder in your workspace

In [27]:
base_url = "https://eoresults.esa.int"

item_to_be_downloaded = 3
target = items[item_to_be_downloaded]

output_dir = f"downloads/{target.id}"
os.makedirs(output_dir, exist_ok=True)

assets_total=len(target.assets.items())
assets_current=0
for asset_key, asset in target.assets.items():
    filename = os.path.basename(asset.href)
    full_href = urljoin(base_url, asset.href)
    local_path = os.path.join(output_dir, filename)
    assets_current+=1
    print(f"[{assets_current}/{assets_total}] Downloading {filename}...")
    try:
        urlretrieve(full_href, local_path)
    except Exception as e:
        print(f"Failed to download {full_href}. {e}")


[1/181] Downloading S3A_SR_2_TDP_LI_20240402T221721_20240402T222201_20250416T191918_0281_111_001______CNE_GRE_V001.nc...
[2/181] Downloading S3A_SR_2_TDP_LI_20240402T235540_20240403T000124_20250416T191918_0344_111_002______CNE_GRE_V001.nc...
[3/181] Downloading S3A_SR_2_TDP_LI_20240403T133505_20240403T133808_20250416T191918_0183_111_010______CNE_GRE_V001.nc...
[4/181] Downloading S3A_SR_2_TDP_LI_20240403T151352_20240403T151803_20250416T191918_0251_111_011______CNE_GRE_V001.nc...
[5/181] Downloading S3A_SR_2_TDP_LI_20240403T165340_20240403T165700_20250416T191921_0201_111_012______CNE_GRE_V001.nc...
[6/181] Downloading S3A_SR_2_TDP_LI_20240403T183338_20240403T183632_20250416T191921_0174_111_013______CNE_GRE_V001.nc...
[7/181] Downloading S3A_SR_2_TDP_LI_20240403T201315_20240403T201615_20250416T191921_0180_111_014______CNE_GRE_V001.nc...
[8/181] Downloading S3A_SR_2_TDP_LI_20240403T215135_20240403T215609_20250416T191921_0273_111_015______CNE_GRE_V001.nc...
[9/181] Downloading S3A_SR_2_TDP

## Download filtered items <br>
Based on the selection done in the previous cell, download the products to the `downloads` folder in your workspace. You will download here the items which result from further filtering options (by mission type, cycle number, region etc.)

In [14]:


target = filtered[0] if len(filtered) > 0 else None

output_dir = f"downloads/{target.id}"
os.makedirs(output_dir, exist_ok=True)

assets_total=len(target.assets.items())
assets_current=0
for asset_key, asset in target.assets.items():
    filename = os.path.basename(asset.href)
    full_href = urljoin(base_url, asset.href)
    local_path = os.path.join(output_dir, filename)
    assets_current+=1
    print(f"[{assets_current}/{assets_total}] Downloading {filename}...")
    try:
        urlretrieve(full_href, local_path)
    except Exception as e:
        print(f"Failed to download {full_href}. {e}")     

[1/382] Downloading S3A_SR_2_TDP_LI_20240306T230712_20240306T231832_20250416T220404_0681_110_001______CNE_ANT_V001.nc...
[2/382] Downloading S3A_SR_2_TDP_LI_20240307T004812_20240307T005847_20250416T220404_0635_110_002______CNE_ANT_V001.nc...
[3/382] Downloading S3A_SR_2_TDP_LI_20240307T022854_20240307T024314_20250416T220404_0860_110_003______CNE_ANT_V001.nc...
[4/382] Downloading S3A_SR_2_TDP_LI_20240307T041025_20240307T042151_20250416T220404_0686_110_004______CNE_ANT_V001.nc...
[5/382] Downloading S3A_SR_2_TDP_LI_20240307T055147_20240307T060142_20250416T220404_0595_110_005______CNE_ANT_V001.nc...
[6/382] Downloading S3A_SR_2_TDP_LI_20240307T073258_20240307T074215_20250416T220405_0557_110_006______CNE_ANT_V001.nc...
[7/382] Downloading S3A_SR_2_TDP_LI_20240307T091451_20240307T092256_20250416T220405_0485_110_007______CNE_ANT_V001.nc...
[8/382] Downloading S3A_SR_2_TDP_LI_20240307T105152_20240307T110308_20250416T220405_0675_110_008______CNE_ANT_V001.nc...
[9/382] Downloading S3A_SR_2_TDP

In [28]:
base_url = "https://eoresults.esa.int"
for index, item in enumerate(filtered, 2):
    output_dir = f"filtered/{item.id}"
    os.makedirs(output_dir, exist_ok=True)

    assets_total = len(item.assets.items())
    assets_current = 0

    for asset_key, asset in item.assets.items():
        filename = os.path.basename(asset.href)
        full_href = urljoin(base_url, asset.href)
        local_path = os.path.join(output_dir, filename)

        assets_current += 1
        print(f"[{index}] [{assets_current}/{assets_total}] Downloading {filename} for item {item.id}...")

        try:
            urlretrieve(full_href, local_path)
        except Exception as e:
            print(f"Failed to download {full_href}. {e}")

print(f"Downloaded assets for {len(filtered)} items.")

[2] [1/385] Downloading S3B_SR_2_TDP_LI_20240412T230927_20240412T232048_20250416T232530_0681_092_001______CNE_ANT_V001.nc for item sentinel-3b-antarctica-cycle092...
[2] [2/385] Downloading S3B_SR_2_TDP_LI_20240413T005024_20240413T010101_20250416T232530_0637_092_002______CNE_ANT_V001.nc for item sentinel-3b-antarctica-cycle092...
[2] [3/385] Downloading S3B_SR_2_TDP_LI_20240413T023106_20240413T024530_20250416T232530_0864_092_003______CNE_ANT_V001.nc for item sentinel-3b-antarctica-cycle092...
[2] [4/385] Downloading S3B_SR_2_TDP_LI_20240413T041238_20240413T042405_20250416T232530_0687_092_004______CNE_ANT_V001.nc for item sentinel-3b-antarctica-cycle092...
[2] [5/385] Downloading S3B_SR_2_TDP_LI_20240413T055410_20240413T060357_20250416T232531_0588_092_005______CNE_ANT_V001.nc for item sentinel-3b-antarctica-cycle092...
[2] [6/385] Downloading S3B_SR_2_TDP_LI_20240413T073521_20240413T074430_20250416T232531_0549_092_006______CNE_ANT_V001.nc for item sentinel-3b-antarctica-cycle092...
[2] 

## Create an archive of products downloaded 

Create an archive of the products downloaded to your workspace and save them in .zip format to make them compressed

In [15]:
# Create an archive of downloaded products 
zip_path = shutil.make_archive(output_dir, 'zip', root_dir=output_dir)
print(f"Created ZIP archive: {zip_path}")

Created ZIP archive: C:\Temp\WPy64-31241\notebooks\downloads\sentinel-3a-antarctica-cycle110.zip
